# Computación Avanzada y sus Aplicaciones a Ingeniería

### Máster Universitario en Ingeniería Informática


# Práctica 3 - Parte II - Clasificación

En esta práctica afrontaremos un problema de clasificación con la librería de Spark MLib.

Ten en cuenta que una vez tengas en marcha Spark, podrás visualizar la evolución de cada trabajo de Spark en  <http://localhost:4040>

En caso de estar utilizando pySpark, **NO** es necesario inicializar el `SparkSession`, es decir, **no** ejecutar la siguiente celda

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("Ejemplo pySparkSQL") \
    .config("spark.sql.warehouse.dir", "file:///D:/tmp/spark-warehouse") \
    .getOrCreate()

sc = spark.sparkContext


En caso de estar usando pySpark, ejecutar el siguiente comando o inciiar pyspark con 

`pyspark --conf spark.sql.warehouse.dir=file:///D:/tmp/spark-warehouse`

Otros imports necesarios:

In [2]:
%matplotlib inline 
import matplotlib.pyplot as plt
from test_helper import Test
from pyspark.sql.functions import *
from pyspark.sql import Row

# Construcción y ajuste de los parámetros de una Pipeline

Generalmente, una pipeline de ML incluye una serie de fases como son el preprocesamiento, la extracción de características, el ajuste del modelo  y la validación.

Por ejemplo, para clasificar documentos de texto tendríamos la segmentación/limpieza, extracción de características y el entrenamiento del modelo con validación cruzada para el ajuste de parámetros.

Aunque existen muchas librerías para cada fase, trabajar con todas no suele ser demasiado fácil, especialmente cuando trabajamos con datasets grandes. Aquí es donde ML de Spark aporta su granito de arena.

En esta práctica vamos a ver como afrontar un problema de clasificación de textos simple como ejemplo de una pipeline de ML para clasificación con Spark MLlib.

In [3]:
# Imports necesarios de `spark.ml`.
from pyspark.ml import *
from pyspark.ml.classification import *
from pyspark.ml.feature import *
from pyspark.ml.param import *
from pyspark.ml.tuning import *
from pyspark.ml.evaluation import *

### El dataset "20 Newsgroups"

En este caso vamos a trabajar con una versión simplificada del dataset 20 newsgroups. Este dataset tiene una colección de artículos de noticias clasificadas en 20 grupos diferentes.

El dataset original se pueden encontrar en  https://archive.ics.uci.edu/ml/datasets/Twenty+Newsgroups.

Para simplificar la tarea, el que podéis encontrar en las prácticas es una versión simplificada del dataset, donde los 20 grupos los dejamos en 2. Trataremos de saber si un artículo está relacionado con ciencia o no.

El dataset que tenéis disponible está ya disponible para leerse como un DataFrame (en el original es necesario leer cada fichero y transformarlo ligeramente). El formato en el que está almacenado es Parquet, el formato binario por defecto utilizado por Spark.

Descargar el fichero y dejarlo en datos/20newsgropuBinaryFiltered/

In [4]:
dbfs_dir = "./datos/20newsgropuBinaryFiltered/"
import os
print "Ficheros en la carpeta:"
os.listdir(dbfs_dir)

Ficheros en la carpeta:


['.part-r-00000-ef9ac4e4-a6ba-49ff-817a-eb473f8c07de.snappy.parquet.crc',
 '.part-r-00001-ef9ac4e4-a6ba-49ff-817a-eb473f8c07de.snappy.parquet.crc',
 '.part-r-00002-ef9ac4e4-a6ba-49ff-817a-eb473f8c07de.snappy.parquet.crc',
 '.part-r-00003-ef9ac4e4-a6ba-49ff-817a-eb473f8c07de.snappy.parquet.crc',
 '._SUCCESS.crc',
 'part-r-00000-ef9ac4e4-a6ba-49ff-817a-eb473f8c07de.snappy.parquet',
 'part-r-00001-ef9ac4e4-a6ba-49ff-817a-eb473f8c07de.snappy.parquet',
 'part-r-00002-ef9ac4e4-a6ba-49ff-817a-eb473f8c07de.snappy.parquet',
 'part-r-00003-ef9ac4e4-a6ba-49ff-817a-eb473f8c07de.snappy.parquet',
 '_SUCCESS']

In [5]:
Test.assertEquals(len(os.listdir(dbfs_dir)), 10, 'La carpeta contiene 10 ficheros')

1 test passed.


Utiliza `spark.read.parquet` para leer la carpeta.

Posteriormente, utilizando [randomSplit](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.randomSplit) divide el DataFrame en el conjunto de training (60%) y el conjunto de test (40%), utilizando la semilla incluida.

Finalmente, cachea los dos DataFrames.

In [6]:
#df = spark.read.format("csv").options(header=True).load("datos/Bike-Sharing-Dataset/hour.csv")
df = spark.read.parquet(dbfs_dir)
seed = 12418L
(training, test) = df.randomSplit(weights=[6.0, 4.0], seed=seed)

training.cache()
test.cache()

DataFrame[label: int, topic: string, id: string, text: string]

In [10]:
Test.assertEquals(training.count(), 11931,  "Número de ejemplos en training incorrecto")
Test.assertEquals(test.count(), 8066, "Número de ejemplos en test incorrecto")
Test.assertEquals(training.is_cached,True, 'Training no cacheado')
Test.assertEquals(test.is_cached, True, 'Test no cacheado')

1 test passed.
1 test passed.
1 test passed.
1 test passed.


Vamos a ver qué forma tiene el DataFrame que tenemos disponible.

In [11]:
training.show(10)

+-----+-----------+-----+--------------------+
|label|      topic|   id|                text|
+-----+-----------+-----+--------------------+
|    0|alt.atheism|49960|From: mathew <mat...|
|    0|alt.atheism|51119|From: I3150101@db...|
|    0|alt.atheism|51120|From: mathew <mat...|
|    0|alt.atheism|51122|From: I3150101@db...|
|    0|alt.atheism|51124|From: I3150101@db...|
|    0|alt.atheism|51127|From: keith@cco.c...|
|    0|alt.atheism|51128|From: keith@cco.c...|
|    0|alt.atheism|51129|From: keith@cco.c...|
|    0|alt.atheism|51134|From: bobbe@vice....|
|    0|alt.atheism|51136|From: bobbe@vice....|
+-----+-----------+-----+--------------------+
only showing top 10 rows



Podemos explorar el dataset estudiando la distribución de tópicos existente.

Para ello crea una consulta que agrupe las noticias por `topic` y cuente cuántas noticias hay en cada uno. Almacena esta información en un DataFrame `topicCount` y muéstrala con show().

In [12]:
topicCount= training.groupBy('topic').count()
topicCount.show()

+--------------------+-----+
|               topic|count|
+--------------------+-----+
|      comp.windows.x|  597|
|        misc.forsale|  596|
|    rec.sport.hockey|  583|
|  rec.sport.baseball|  619|
|  talk.politics.guns|  609|
|comp.os.ms-window...|  596|
|  talk.politics.misc|  608|
|comp.sys.ibm.pc.h...|  622|
|       comp.graphics|  576|
|soc.religion.chri...|  608|
|comp.sys.mac.hard...|  599|
|  talk.religion.misc|  594|
|talk.politics.mid...|  608|
|     rec.motorcycles|  599|
|           rec.autos|  593|
|         alt.atheism|  598|
|     sci.electronics|  573|
|           sci.space|  574|
|             sci.med|  599|
|           sci.crypt|  580|
+--------------------+-----+



In [13]:
Test.assertEquals(sorted(topicCount.collect()), [(u'alt.atheism', 598), (u'comp.graphics', 576), (u'comp.os.ms-windows.misc', 596), 
                                                 (u'comp.sys.ibm.pc.hardware', 622), (u'comp.sys.mac.hardware', 599),
                                                 (u'comp.windows.x', 597), (u'misc.forsale', 596), (u'rec.autos', 593), 
                                                 (u'rec.motorcycles', 599), (u'rec.sport.baseball', 619), (u'rec.sport.hockey', 583),
                                                 (u'sci.crypt', 580), (u'sci.electronics', 573), (u'sci.med', 599), (u'sci.space', 574),
                                                 (u'soc.religion.christian', 608), (u'talk.politics.guns', 609), 
                                                 (u'talk.politics.mideast', 608), (u'talk.politics.misc', 608), 
                                                 (u'talk.religion.misc', 594)], "Conteo por tópic incorrecto")

1 test passed.


Dado que nuestro objetivo es predecir la etiqueta `label`, es decir, si el artículo está relacionado con la ciencia o no, es interesante observar la distribución de ejemplos por etiqueta.

Utiliza para ello el método `groupBy` seguido de `count`para obtener el número de ejemplos por etiqueta que tenemos en el dataset de train. Almacena el resultado en el DataFrame `labelCount` y utiliza show() para mostrarlo.

In [14]:
labelCount = training.groupBy('label').count()
labelCount.show()

+-----+-----+
|label|count|
+-----+-----+
|    1| 2326|
|    0| 9605|
+-----+-----+



In [15]:
Test.assertEquals(sorted(labelCount.collect()), [(0, 9605),(1, 2326)], "Conteo por tópic incorrecto")

1 test passed.


### Construcción de la Pipeline para clasificar artículos de noticias

Nuestra pipline tendrá las siguientes etapas:

1. **RegexTokenizer**, tokeniza cada artículo a secuencias de palabras con un patrón de expresiones regulares,
2. **HashingTF**, mapea las secuencias de palabras producidas por RegexTokenizer a vectores de características dispersos usando hashing (no nos dentendremos en ver cómo lo hace, simplemente sabemos que coge listas de palabras y nos da un vectore de características),
3. **LogisticRegression**, entrena un modelo de regresión logísticas usando los vectores de características y las etiquetas del conjunto de entrenamiento.

<img src="http://spark.apache.org/docs/latest/img/ml-Pipeline.png" style="width: 800px;"/>

In [16]:
# Debemos construir cada fase de la Pipeline con sus parámetros y finalmente crear la Pipeline

# Tokenizer: columna de entrada = test, columna de salida = words y patrón a buscar s+ (dividimos por espacios)
tokenizer = RegexTokenizer(inputCol="text", outputCol="words", pattern="s+")

# HashingTF: indicamos la columna de entrada como la de salida del tokenizer, la de salida = features y el número de caraccterísticas a obtener (5000)
hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="features", numFeatures=5000)

# Regresión logística con 20 iteraciones y parámetro de regularización = 0.01
lr = LogisticRegression(maxIter=20, regParam=0.01)

# Creamos la pipline de ML como una lista de fases
pipeline = Pipeline(stages=[tokenizer, hashingTF, lr])

In [18]:
# Una vez construida la pipeline, podemos entrenar el modelo con el conjunto de training
# Utiliza fit para ajustar el modelo al training
model = pipeline.fit(training)

### Comprobar y evaluar las predicciones

Una vez obtenido el PipelineModel, queremos saber cómo se comporta.
Primero lo haremos visualizando las etiquetas predichas.

In [19]:
# Utiliza transform para predecir los resultados sobre el conjunto de training
prediction = model.transform(training)

# Muestra las etiquetas predichas junto con las reales y el testo (prediction, label y text)
# Show the predicted labels along with true labels and raw texts.
prediction.select("prediction", "label", "text").show()

+----------+-----+--------------------+
|prediction|label|                text|
+----------+-----+--------------------+
|       0.0|    0|From: mathew <mat...|
|       0.0|    0|From: I3150101@db...|
|       0.0|    0|From: mathew <mat...|
|       0.0|    0|From: I3150101@db...|
|       0.0|    0|From: I3150101@db...|
|       0.0|    0|From: keith@cco.c...|
|       0.0|    0|From: keith@cco.c...|
|       0.0|    0|From: keith@cco.c...|
|       0.0|    0|From: bobbe@vice....|
|       0.0|    0|From: bobbe@vice....|
|       0.0|    0|From: smullins@ci...|
|       0.0|    0|From: halat@pooh....|
|       0.0|    0|From: dgraham@bme...|
|       0.0|    0|From: keith@cco.c...|
|       0.0|    0|From: livesey@sol...|
|       0.0|    0|From: livesey@sol...|
|       0.0|    0|From: anthropo@ca...|
|       0.0|    0|From: keith@cco.c...|
|       0.0|    0|From: acooper@mac...|
|       0.0|    0|From: Nanci Ann M...|
+----------+-----+--------------------+
only showing top 20 rows



Parece que los resultados sobre training son buenos. Pero vamos a ver el resultado cuantitativo.

In [21]:
# Creamos un evaluador para clasificación binaria usando el área bajo la curva ROC
evaluator = BinaryClassificationEvaluator(metricName="areaUnderROC")

# Utiliza el evaluador creado con el método evaluate para obtener el AUC del modelo sobre train
evaluator.evaluate(prediction)

0.9997319082252888

El resultado en training es prácticamente perfecto, pero esto suele ser muchas veces una pista de que estamos sobrentrenando. Veamos el resultado sobre test.

In [36]:
# Utiliza el método evaluate del evaluador y el método transform del modelo para obtener las predicciones sobre test y posteriomente evaluarlas
evaluator.evaluate(model.transform(test))

0.9613702836430874

El AUC sobre test es mucho más pequeño.
Parece que tenemos algún problema más aparte del sobreentrenamiento. 
Estudiemos las fases establecidas.

## Comprobación de la Pipeline

Las predicciones de la pipline tienen también resultados intermedios de cada fase:
* "words" del tokenizer,
* "features" del hashing ,
* "prediction", "probability", y "rawPredictions" de la regresión logística.

Veamos el esquema de "prediction".

In [23]:
prediction.printSchema()

root
 |-- label: integer (nullable = true)
 |-- topic: string (nullable = true)
 |-- id: string (nullable = true)
 |-- text: string (nullable = true)
 |-- words: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = true)



Podemos mirar todas las columnas usando show() sobre prediction.

¿Qué no nos cuadra?

In [24]:
prediction.show(10)

+-----+-----------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|label|      topic|   id|                text|               words|            features|       rawPrediction|         probability|prediction|
+-----+-----------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|    0|alt.atheism|49960|From: mathew <mat...|[from: mathew <ma...|(5000,[10,18,20,4...|[31.5751023364328...|[0.99999999999998...|       0.0|
|    0|alt.atheism|51119|From: I3150101@db...|[from: i3150101@d...|(5000,[12,14,44,4...|[10.0912542735155...|[0.99995856131582...|       0.0|
|    0|alt.atheism|51120|From: mathew <mat...|[from: mathew <ma...|(5000,[13,17,20,5...|[2.60344796911761...|[0.93108315569236...|       0.0|
|    0|alt.atheism|51122|From: I3150101@db...|[from: i3150101@d...|(5000,[15,44,46,6...|[9.05806892958553...|[0.99988356593029...|       0.0|
|    0

Si nos fijamos bien, la columna "words" debería tener un array de strings con palabras, pero sin embargo vemos que algo no ha ido bien. Por lo que parece que el tokenizer no está funcionando

Vamos a usar `explainParams` sobre el tokenizer para ver los parámetros establecidos y su documentación.

In [25]:
print tokenizer.explainParams()

gaps: whether regex splits on gaps (True) or matches tokens (False) (default: True)
inputCol: input column name. (current: text)
minTokenLength: minimum token length (>= 0) (default: 1)
outputCol: output column name. (default: RegexTokenizer_469a981a37af25f5b39b__output, current: words)
pattern: regex pattern (Java dialect) used for tokenizing (default: \s+, current: s+)
toLowercase: whether to convert all characters to lowercase before tokenizing (default: True)


¡Cuidado! Nos hemos olvidado de la contrabarra en la expresión regular... Debe ser "\s+" y no "s+". Vamos a corregirlo...

In [31]:
# Utiliza el método setPattern de tokenizer para poner el patrónSet the value of "pattern" back to "\s+",
# necesitarás incluir una doble contrabarra "\\s+"
tokenizer.setPattern(value="\\s+")
print tokenizer.explainParams()

gaps: whether regex splits on gaps (True) or matches tokens (False) (default: True)
inputCol: input column name. (current: text)
minTokenLength: minimum token length (>= 0) (default: 1)
outputCol: output column name. (default: RegexTokenizer_469a981a37af25f5b39b__output, current: words)
pattern: regex pattern (Java dialect) used for tokenizing (default: \s+, current: \s+)
toLowercase: whether to convert all characters to lowercase before tokenizing (default: True)


In [32]:
# Entrenamos el modelo de nuevo
model = pipeline.fit(training)

In [33]:
# Comprobamso las predicciones y que las palabras tiene buena pinta
prediction = model.transform(training)
prediction.show(10)

+-----+-----------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|label|      topic|   id|                text|               words|            features|       rawPrediction|         probability|prediction|
+-----+-----------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|    0|alt.atheism|49960|From: mathew <mat...|[from:, mathew, <...|(5000,[1,21,23,30...|[26.6746362960874...|[0.99999999999739...|       0.0|
|    0|alt.atheism|51119|From: I3150101@db...|[from:, i3150101@...|(5000,[19,76,78,8...|[2.91331462750431...|[0.94850071540962...|       0.0|
|    0|alt.atheism|51120|From: mathew <mat...|[from:, mathew, <...|(5000,[91,143,153...|[9.55014157499995...|[0.99992881388436...|       0.0|
|    0|alt.atheism|51122|From: I3150101@db...|[from:, i3150101@...|(5000,[2,9,19,28,...|[6.80827698101851...|[0.99889662428421...|       0.0|
|    0

In [34]:
# Evaluamos el modelo en train y en test
evaluator.evaluate(prediction)

0.9998549542706467

In [35]:
evaluator.evaluate(model.transform(test))

0.9613702836430876

Ahora sí que parece que funciona mejor, aunque seguimos teniendo algo de sobrentrenamiento.
Para mejorar el error de generalización podemos ajustar los parámetros de la Pipeline.

## Ajuste de parámetros mediante validación cruzada

Podemos usar la validación cruzada en MLlib mediante `CrossValidator`.

Se toma una lista de combinaciones de parámetros y una medida de evaluación y él automáticamente busca la mejor combinación mediante validación cruzada.

Consulta la siguiente documentación para preparar la validación cruzada.

[CrossValidator](http://spark.apache.org/docs/2.0.2/api/python/pyspark.ml.html#pyspark.ml.tuning.CrossValidator)

[ParamGridBuilder](http://spark.apache.org/docs/2.0.2/api/python/pyspark.ml.html#pyspark.ml.tuning.ParamGridBuilder)

In [40]:
# Las combinaciones de parámetros se generan como todas las posibles combinaciones entre los diferentes parámetros establecidos
# Para simplificarlo, solo usaremos diferentes parámetros de hashing TF y del parámetro de regularización de la regresión
# Utiliza ParamGridBuilder para considerar los siguientes parámetros
# hashingTF.numFeatures = [1000, 10000]
# lr.regParam = [0.05, 0.2]

paramGrid = ParamGridBuilder() \
    .addGrid(hashingTF.numFeatures, [1000, 10000]) \
    .addGrid(lr.regParam, [0.05, 0.2]) \
    .build()

# Creamos un CrossValidator para ajustar la pipeline
# Utilizamos como estimator la pipeline, como evaluator el evaluator ya definido y como parámetros paramGrid.
# Para que tarde menos utiliza solo 2 particiones

cv = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator,
                          numFolds=2)

Ajustar un modelo de validación cruzada funciona igual que hacerlo con la Pipeline. Llevará más tiempo porque se ajustan muchos más modelos para elegir el mejor.

In [41]:
# Utiliza fit sobre cv para obtener el modelo entrenado con el conjunto de training
cvModel = cv.fit(training)

Veamos los resultados que obtenemos ahora

In [42]:
evaluator.evaluate(cvModel.transform(training))

0.9991227206380309

In [43]:
evaluator.evaluate(cvModel.transform(test))

0.975911309387619

¡Hemos mejorado en test!
Solo hemos probado unas pocas combinaciones de parámetros. Podríamos realizar muchos más experimentos para mejorar todavía más los resultados.